# **<span style="color: #0098cd;">Proyecto: Named-Entity Recognition</span>**


## **Objetivos** 

## **Descripción**

### **Carga y preprocesamiento del texto**

In [7]:
#!pip install spacy
#!pip install chardet
#python -m spacy download es_core_news_md

import pathlib 
import spacy
import pandas as pd
from spacy import displacy
import csv
import chardet

#### **Carga del modelo y detección de codificación**

In [8]:
try:
    nlp = spacy.load("es_core_news_md")  # Cargo el modelo de Spacy para español
except OSError:
    print("El modelo 'es_core_news_md' no está instalado.")

In [9]:
filename = "Dataset.csv"

with open(filename, "rb") as f:
    rawdata = f.read(10000)           # Leo los primeros 10.000 bytes
    result = chardet.detect(rawdata)  # Detecto la codificación
    print(result)                     # Muestro la codificación detectada

{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}


#### **Carga del dataset**

In [15]:
data = pd.read_csv(filename, delimiter=';', encoding='utf-8', encoding_errors='ignore', low_memory=False)  # Cargo el archivo CSV ignorando carácteres raros.

El dataset tiene una codificación UTF-8 con un 99% de confianza, pero contiene algunos carácteres extraños que representan una cantidad despreciable de carácteres, decido ignorarlos.

### **Primera parte de preguntas**

Preguntas a responder:
1. ¿Cuántos registros contiene el corpus?
2. ¿Cuántas palabras totales hay en los comentarios del corpus?
3. ¿Cuál el número promedio de palabras en cada comentario?

#### **<span style="color: #0098cd;">1. ¿Cuántos registros contiene el corpus? ¿Hay nulos?</span>**

In [16]:
regTotal = data.shape[0]        # Obtengo el número de registros del dataset
#regTotal = len(data)           # Otra opción para obtener el número de registros
regNulos = data.isnull().sum()  # Obtengo el número de nulos en cada columna

print(f"Número de registros en total: {regTotal}")
print("_"*37)
print(f"Valores nulos por columna: ")
print(regNulos)
print("_"*40)

Número de registros en total: 574915
_____________________________________
Valores nulos por columna: 
MEDIO                        0
SOPORTE                     34
URL                        148
TIPO DE MENSAJE            152
CONTENIDO A ANALIZAR       152
INTENSIDAD                 273
TIPO DE ODIO            562619
TONO HUMORISTICO        574769
MODIFICADOR             574562
Unnamed: 9              574912
Unnamed: 10             574912
Unnamed: 11             574913
Unnamed: 12             574913
Unnamed: 13             574913
Unnamed: 14             574913
Unnamed: 15             574913
dtype: int64
________________________________________


In [ ]:
data = data.drop(columns=['Unnamed: 9', 'Unnamed: 10',         # Elimino columnas innecesarias con mayoría de nulos
                          'Unnamed: 11', 'Unnamed: 12', 
                          'Unnamed: 13', 'Unnamed: 14', 
                          'Unnamed: 15', 'TONO HUMORISTICO', 
                          'MODIFICADOR', 'TIPO DE ODIO'])


regNoNulos = data.dropna(subset=['CONTENIDO A ANALIZAR', 'INTENSIDAD']) # Elimino los registros con nulos en las columnas 'CONTENIDO A ANALIZAR' e 'INTENSIDAD'
print(f"Registros tras filtrar los nulos: {len(regNoNulos)}")
print("_"*40)

regNulos = data.isnull().sum()              # Compruebo el número de nulos en cada columna de nuevo
print(f"Valores nulos por columna: ")
print(regNulos)

Registros tras filtrar los nulos: 574642
________________________________________
Valores nulos por columna: 
MEDIO                     0
SOPORTE                  34
URL                     148
TIPO DE MENSAJE         152
CONTENIDO A ANALIZAR    152
INTENSIDAD              273
dtype: int64


Elimino las columnas con mayoría de nulos ya que no aportarán información relevante y dado que el análisis se basa en el contenido de los mensajes y su "intensidad" de odio, elimino también los registros donde faltan estos valores.

#### **<span style="color: #0098cd;">2. ¿Cuántas palabras totales hay en los comentarios del corpus?</span>**

In [34]:
# Selecciono una muestra de 20.000 registros aleatorios para pruebas más rápidas
lines_number = 20000                                      # Para cargar solo un número limitado de líneas
data_sample = data.sample(lines_number, random_state=7)   # Tomo una muestra para pruebas más rápidas

print(f"Registros en la muestra: {len(data_sample)}")     # Verifico que se han cogido los 20000 registros.
print("-"*30)

regNulos = data_sample.isnull().sum()                     # Compruebo el número de nulos en la muestra
print(f"Valores nulos por columna en la muestra: ")
print(regNulos)

Registros en la muestra: 20000
------------------------------
Valores nulos por columna en la muestra: 
MEDIO                    0
SOPORTE                  3
URL                      4
TIPO DE MENSAJE          5
CONTENIDO A ANALIZAR     4
INTENSIDAD              11
dtype: int64


In [51]:
# Función para contar las palabras sin signos de puntuación
#----------------------------------------------------------
# is_alpha verifica que sea alfabético, es decir, que no sean signos de puntuación como -> . , ; : ! ? etc.
# is_punct verifica que es un signo de puntuación
# is_space verifica que es un espacio
# Por tanto, si el token es alfabético y no es un signo de puntuación ni un espacio, lo considero una palabra.
def CuentaPalab(mensaje):
    doc = nlp(mensaje)
    palabras = [token.text for token in doc if token.is_alpha and not token.is_punct and not token.is_space]
    return len(palabras)

# Ejemplo de uso de la función CuentaPalab
ejemplo = "Hola,  ¿cómo estás?   ¿Todo     bien? ¿Todo correcto?."
print("Conteo con SpaCy:", CuentaPalab(ejemplo)) # Debería dar 7

Conteo con SpaCy: 7


In [53]:
# Cuento las palabras teniendo en cuenta los signos de puntuación
ComentMuestra = data_sample[data_sample["TIPO DE MENSAJE"] == "COMENTARIO"]               # Cojo solo los que cuyo TIPO DE MENSAJE es COMENTARIO
palabTotal = ComentMuestra["CONTENIDO A ANALIZAR"].apply(lambda x: len(str(x).split()))   # Cuento el número total de palabras en los comentarios
SumapalabTotal = palabTotal.sum()
print(f"Palabras en total (con todo) en comentarios: {SumapalabTotal}")
print("-"*51)

# Cuento las palabras sin tener en cuenta los signos de puntuación ni espacios
AlfTotal = ComentMuestra["CONTENIDO A ANALIZAR"].apply(CuentaPalab).sum()                 # Aplico la función a cada comentario y voy sumando
print(f"Palabras en total (solo alfabéticas) en los comentarios de la muestra: {AlfTotal}")
print("-"*77)


Palabras en total (con todo) en comentarios: 524302
---------------------------------------------------
Palabras en total (solo alfabéticas) en los comentarios de la muestra: 505975
-----------------------------------------------------------------------------


Debido a los tiempos de procesamiento y a la carga computacional, para esta pregunta y para las posteriores, he seleccionado una muestra de 20.000 registros añadiendo un estado fijo para asegurar que siempre escoja los mismos registros en posteriores ejecuciones y así no distorsionar los resultados.

He considerado que los signos de puntuación no son palabras.

#### **<span style="color: #0098cd;">3. ¿Cuál el número promedio de palabras en cada comentario?</span>**

In [54]:
PalabrasPorComentario = ComentMuestra["CONTENIDO A ANALIZAR"].apply(CuentaPalab) 

AlfProm = PalabrasPorComentario.mean()           # Calculo el promedio
print(f"Promedio de palabras alfabéticas por comentario de la muestra: {AlfProm:.2f}")

Promedio de palabras alfabéticas por comentario de la muestra: 44.02


### **Segunda parte de preguntas**

Considerando dos grupos de comentarios (odio y no odio):